In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from math import ceil

In [ ]:
# Load and preprocess data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/chicken_photos/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/chicken_photos/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

# Calculate steps per epoch and validation steps
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size

# Load pre-trained VGG16 model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_base.trainable = False  # Freeze the convolutional base

# Add custom layers
vgg16_model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Load pre-trained ResNet50 model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_base.trainable = False  # Freeze the convolutional base

# Add custom layers
resnet50_model = models.Sequential([
    resnet50_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile VGG16 model
vgg16_model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

# Compile ResNet50 model
resnet50_model.compile(loss='binary_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])

# Train the VGG16 model
vgg16_history = vgg16_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps)

# Train the ResNet50 model
resnet50_history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps)


Found 779 images belonging to 2 classes.
Found 193 images belonging to 2 classes.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
24/24 [==============================] - 712s 29s/step - loss: 0.4888 - accuracy: 0.8849 - val_loss: 0.1331 - val_accuracy: 0.9844
Epoch 2/10
24/24 [==============================] - 676s 28s/step - loss: 0.0213 - accuracy: 0.9920 - val_loss: 0.1705 - val_accuracy: 0.9740
Epoch 3/10
24/24 [==============================] - 742s 31s/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.1492 - val_accuracy: 0.9844
Epoch 4/10
24/24 [==============================] - 684s 29s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1600 - val_accuracy: 0.9844
Epoch 5/10
24/24 [==============================] - 678s 28s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1593 - val_accuracy: 0.9844
Epoch 6/10
24/24 [==============================] - 677s 28s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.1557 - val_accuracy: 0.9844
E

In [ ]:
# Assuming the test images are in a directory 'test_photos/'
test_directory_path = '/content/drive/MyDrive/test_photos/'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_directory_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)  # No need to shuffle test data

# Check if the test generator has loaded any data
if test_generator.n == 0:
    raise ValueError("No images found in test directory")

# Calculate the correct number of steps for the predict method
test_steps = ceil(test_generator.n / test_generator.batch_size)

# Predict with VGG16 model
vgg16_predictions = vgg16_model.predict(test_generator, steps=test_steps)

# Predict with ResNet50 model
resnet50_predictions = resnet50_model.predict(test_generator, steps=test_steps)

# Flatten predictions and truncate to match the number of labels
vgg16_predictions = vgg16_predictions.flatten()[:test_generator.n]
resnet50_predictions = resnet50_predictions.flatten()[:test_generator.n]

# Convert predictions to binary (0 or 1)
vgg16_predictions = [1 if x > 0.5 else 0 for x in vgg16_predictions]
resnet50_predictions = [1 if x > 0.5 else 0 for x in resnet50_predictions]

# True labels
true_labels = test_generator.classes

# Ensure the predictions and labels have the same length
assert len(vgg16_predictions) == len(true_labels), "Mismatch in VGG16 samples"
assert len(resnet50_predictions) == len(true_labels), "Mismatch in ResNet50 samples"

# Evaluate models
print("VGG16 Model Performance:")
print(classification_report(true_labels, vgg16_predictions))

print("ResNet50 Model Performance:")
print(classification_report(true_labels, resnet50_predictions))

Found 332 images belonging to 2 classes.
11/11 [==============================] - 72s 6s/step
VGG16 Model Performance:
              precision    recall  f1-score   support

           0       1.00      0.64      0.78       143
           1       0.79      1.00      0.88       189

    accuracy                           0.85       332
   macro avg       0.89      0.82      0.83       332
weighted avg       0.88      0.85      0.84       332

ResNet50 Model Performance:
              precision    recall  f1-score   support

           0       0.90      0.36      0.52       143
           1       0.67      0.97      0.79       189

    accuracy                           0.71       332
   macro avg       0.78      0.67      0.65       332
weighted avg       0.77      0.71      0.67       332



In [ ]:
print(train_generator.class_indices)

{'chicken': 0, 'non_chicken': 1}
